### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from collections import defaultdict
import json
pio.templates.default = "plotly_dark"

In [3]:
import os 
for dirname, _, filenames in os.walk('/Users/ashishgohil/programming/data analysis/projects/project9'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/ashishgohil/programming/data analysis/projects/project9/.DS_Store
/Users/ashishgohil/programming/data analysis/projects/project9/test.csv
/Users/ashishgohil/programming/data analysis/projects/project9/submission.csv
/Users/ashishgohil/programming/data analysis/projects/project9/train.csv
/Users/ashishgohil/programming/data analysis/projects/project9/Death toll analysis.ipynb
/Users/ashishgohil/programming/data analysis/projects/project9/.ipynb_checkpoints/Death toll analysis-checkpoint.ipynb


### Importing data

In [10]:
filename_train = './train.csv'
filename_test = './test.csv'
# filename_submission = './submission.csv'

In [11]:
train = pd.read_csv(filename_train)
test = pd.read_csv(filename_test)
# submission = pd.read_csv(filename_submission)

In [12]:
train.head(5)
test.head(5)
# submission(5)

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-26
1,2,NaN,Afghanistan,2020-03-27
2,3,NaN,Afghanistan,2020-03-28
3,4,NaN,Afghanistan,2020-03-29
4,5,NaN,Afghanistan,2020-03-30


### SQL Alchemy Engine

In [13]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

train.to_sql('train', con=engine)
test.to_sql('test', con=engine)

print(engine.table_names())

['test', 'train']


### Confirmed cases over time

In [15]:
table = "train"
query = "select Date ,SUM(ConfirmedCases) AS Confirmed FROM {} GROUP BY Date".format(table)
df = pd.read_sql(query, engine)

df.head()

,Date,Confirmed
0,2020-01-22,554.0
1,2020-01-23,653.0
2,2020-01-24,939.0
3,2020-01-25,1432.0
4,2020-01-26,2113.0


In [19]:
fig = px.line(df, x='Date', y='Confirmed', title='Worldwide Confirm Cases Over Time')
fig.show()

fig = px.line(df, x='Date', y='Confirmed',
              title="Worldwide Confirmed Cases (Logarithmic Scale) Over Time",
              log_y=True)

fig.show()

### Cases by Countries

In [22]:
def query_country (country, table):
    query = """
        WITH tmp AS 
        (
            SELECT * FROM {} WHERE Country_Region=\'{}\'
        )
        SELECT Date, SUM(ConfirmedCases) AS confirmed
        FROM tmp
        GROUP BY Date
    """.format(table, country)
    print(query)
    df = pd.read_sql(query, engine)
    return df

results = defaultdict()

table = 'train'
country = 'China'
results[country] = query_country(country, table)

table = 'train'
country = 'US'
results[country] = query_country(country, table)

table = 'train'
country = 'India'
results[country] = query_country(country, table)

table = 'train'
country = 'Italy'
results[country] = query_country(country, table)


        WITH tmp AS 
        (
            SELECT * FROM train WHERE Country_Region='China'
        )
        SELECT Date, SUM(ConfirmedCases) AS confirmed
        FROM tmp
        GROUP BY Date
    

        WITH tmp AS 
        (
            SELECT * FROM train WHERE Country_Region='US'
        )
        SELECT Date, SUM(ConfirmedCases) AS confirmed
        FROM tmp
        GROUP BY Date
    

        WITH tmp AS 
        (
            SELECT * FROM train WHERE Country_Region='India'
        )
        SELECT Date, SUM(ConfirmedCases) AS confirmed
        FROM tmp
        GROUP BY Date
    

        WITH tmp AS 
        (
            SELECT * FROM train WHERE Country_Region='Italy'
        )
        SELECT Date, SUM(ConfirmedCases) AS confirmed
        FROM tmp
        GROUP BY Date
    


In [23]:
table = 'train'
excluded_countries = ('China', 'Italy', 'US', 'India')

query = """
    WITH tmp AS
    (
        SELECT * FROM {} WHERE Country_Region NOT IN {}
    )
    SELECT Date, SUM(ConfirmedCases) AS confirmed
    FROM tmp
    GROUP BY Date
""".format(table, excluded_countries)

print(query)

df = pd.read_sql(query, engine)

results['Rest of the World'] = df


    WITH tmp AS
    (
        SELECT * FROM train WHERE Country_Region NOT IN ('China', 'Italy', 'US', 'India')
    )
    SELECT Date, SUM(ConfirmedCases) AS confirmed
    FROM tmp
    GROUP BY Date



In [24]:
colors = ('#F61067', '#91C4F2', '#6F2DBD', '#00FF00', '#FFDF64')

for c, (country, df) in zip(colors, results.items()):
    fig = px.line(results[country], x='Date', y='confirmed',
                 title="Confirmed Cases in {} Over Time".format(country),
                 color_discrete_sequence=[c],
                 height=500)
    fig.show()

In [39]:
table = 'train'

query = """
    SELECT t.Date, t.Country_Region AS country, SUM(t.ConfirmedCases) as confirmed
    FROM {0} AS t
    INNER JOIN 
        (SELECT max(Date) AS MaxDate, Country_Region FROM {0} GROUP BY Country_Region
    ) tmp 
    ON tmp .MaxDate=t.Date and tmp.Country_Region = t.Country_Region
    GROUP BY t.Country_Region
    """.format(table)

latest_grouped = pd.read_sql(query, engine)
latest_grouped.head()

,Date,country,confirmed
0,2020-04-07,Afghanistan,423.0
1,2020-04-07,Albania,383.0
2,2020-04-07,Algeria,1468.0
3,2020-04-07,Andorra,545.0
4,2020-04-07,Angola,17.0


In [40]:
fig = px.choropleth(latest_grouped, locations = 'country',
                   locationmode='country names', color='confirmed',
                   hover_name='country', range_color=[1, 5000],
                   color_continuous_scale='peach',
                   title='Countries with confirmed cases' 
                   )
fig.show()


### Cases in European Countries

In [42]:
table = 'train'

europe = list(['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland',
               'Italy', 'Latvia','Luxembourg','Lithuania','Malta','Norway','Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia',
               'Spain', 'Sweden', 'United Kingdom', 'Iceland', 'Russia', 'Switzerland', 'Serbia', 'Ukraine', 'Belarus',
               'Albania', 'Bosnia and Herzegovina', 'Kosovo', 'Moldova', 'Montenegro', 'North Macedonia'])

europe_grouped_latest = latest_grouped[latest_grouped['country'].isin(europe)]
europe_grouped_latest.head()

,Date,country,confirmed
1,2020-04-07,Albania,383.0
9,2020-04-07,Austria,12639.0
15,2020-04-07,Belarus,861.0
16,2020-04-07,Belgium,22194.0
21,2020-04-07,Bosnia and Herzegovina,764.0


In [43]:
fig = px.choropleth(europe_grouped_latest, locations='country',
                   locationmode='country names', color='confirmed',
                   hover_name='country', range_color=[1, 2000],
                   color_continuous_scale='portland', 
                    title='European Countries with Confirmed Cases', scope='europe', height=800)

fig.show()

### Confirmed cases worldwide

In [44]:
fig = px.bar(latest_grouped.sort_values('confirmed', ascending=False)[:20][::-1],
x='confirmed', y='country',
title='Confirmed Cases World Wide', text='confirmed', height=1000, orientation='h')

fig.show()

### Worldwide analysis

In [46]:
table = 'train'

query = """
    SELECT Date, Country_Region AS country, SUM(ConfirmedCases) AS confirmed, SUM(Fatalities) AS deaths
    FROM {}
    GROUP BY Date, Country_Region
""".format(table)

print(query)

formated_gdf = pd.read_sql(query, engine)


formated_gdf['size'] = formated_gdf['confirmed'].pow(0.3)

display(formated_gdf)


    SELECT Date, Country_Region AS country, SUM(ConfirmedCases) AS confirmed, SUM(Fatalities) AS deaths
    FROM train
    GROUP BY Date, Country_Region



,Date,country,confirmed,deaths,size
0,2020-01-22,Afghanistan,0.0,0.0,0.000000
1,2020-01-22,Albania,0.0,0.0,0.000000
2,2020-01-22,Algeria,0.0,0.0,0.000000
3,2020-01-22,Andorra,0.0,0.0,0.000000
4,2020-01-22,Angola,0.0,0.0,0.000000
...,...,...,...,...,...
13855,2020-04-07,Venezuela,165.0,7.0,4.626421
13856,2020-04-07,Vietnam,249.0,0.0,5.234314
13857,2020-04-07,West Bank and Gaza,261.0,1.0,5.308749
13858,2020-04-07,Zambia,39.0,1.0,3.001369


In [47]:
fig = px.scatter_geo(formated_gdf, locations='country',
                    locationmode='country names', color='confirmed',
                    size='size', hover_name='country', range_color=[0, 1500],
                    projection='natural earth', animation_frame='Date',
                    title='COVID-19: Spread Over Time', color_continuous_scale="portland")

fig.show()